In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
from delta import *

# warehouse_location points to the default location for managed databases and tables
warehouse_location = 'hdfs://hdfs-nn:9000/warehouse'

builder = SparkSession \
    .builder \
    .appName("Python Spark SQL Hive integration example") \
    .config("spark.sql.warehouse.dir", warehouse_location) \
    .config("hive.metastore.uris", "thrift://hive-metastore:9083") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.jars.packages", "io.delta:delta-core_2.12:2.4.0") \
    .enableHiveSupport() \

spark = spark = configure_spark_with_delta_pip(builder).getOrCreate()

In [ ]:
#############################################################################################################################################################################
######################################## COVID ##############################################################################################################################
#############################################################################################################################################################################

In [19]:
#read hdfs file to dataframe
#
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, regexp_replace

spark = SparkSession.builder.appName("HDFSToDataFrame").getOrCreate()

hdfs_file_path = "hdfs://hdfs-nn:9000/projeto_EDSTD/bronze/Covid.csv"

dfCovid = spark.read.csv(hdfs_file_path, header=True, inferSchema=True)

#eliminar colunas
columns_to_drop = ['Active Cases']
dfCovid = dfCovid.drop(*columns_to_drop)

#rename colunas
dfCovid = dfCovid.withColumnRenamed("Serial Number", "SerialNumber") \
                 .withColumnRenamed("Country", "CountryName") \
                 .withColumnRenamed("Total Cases", "TotalCases") \
                 .withColumnRenamed("Total Deaths", "TotalDeaths") \
                 .withColumnRenamed("Total Recovered", "TotalRecovered") \
                 .withColumnRenamed("Total Test", "TotalTest")

# Remova as vírgulas da coluna 
dfCovid = dfCovid.withColumn("TotalRecovered", regexp_replace(col("TotalRecovered"), ",", ""))
dfCovid = dfCovid.withColumn("TotalTest", regexp_replace(col("TotalTest"), ",", ""))
dfCovid = dfCovid.withColumn("Population", regexp_replace(col("Population"), ",", ""))

# Converter StringType para IntegerType
dfCovid = dfCovid.withColumn("TotalCases", col("TotalCases").cast("integer"))
dfCovid = dfCovid.withColumn("TotalDeaths", col("TotalDeaths").cast("integer"))
dfCovid = dfCovid.withColumn("TotalRecovered", col("TotalRecovered").cast("integer"))
dfCovid = dfCovid.withColumn("TotalTest", col("TotalTest").cast("integer"))
dfCovid = dfCovid.withColumn("Population", col("Population").cast("integer"))

dfCovid.toPandas()

SerialNumber       CountryName  TotalCases  TotalDeaths  TotalRecovered  \
0               1               USA         NaN          NaN     101322779.0   
1               2             India         NaN          NaN      44150289.0   
2               3            France         NaN          NaN      39264546.0   
3               4           Germany         NaN          NaN      37398100.0   
4               5            Brazil         NaN          NaN      35919372.0   
..            ...               ...         ...          ...             ...   
226           227  Diamond Princess       712.0         13.0           699.0   
227           228      Vatican City        29.0          NaN            29.0   
228           229    Western Sahara        10.0          1.0             9.0   
229           230        MS Zaandam         9.0          2.0             7.0   
230           231           Tokelau         5.0          NaN             NaN   

        TotalTest    Population  
0    1.159833e+09  3.348053e+08  
1    9.152658e+08  1.406632e+09  
2    2.714902e+08  6.558452e+07  
3    1.223324e+08  8.388360e+07  
4    6.377617e+07  2.153536e+08  
..            ...           ...  
226           NaN           NaN  
227           NaN  7.990000e+02  
228           NaN  6.261610e+05  
229           NaN           NaN  
230           NaN  1.378000e+03  

[231 rows x 7 columns]

In [20]:
from delta import DeltaTable
from pyspark.sql import SparkSession

# Inicialize a sessão Spark
spark = SparkSession.builder.appName("Write to Delta Lake").getOrCreate()

# Caminho da tabela Delta no HDFS
covid_table = "hdfs://hdfs-nn:9000/projeto_EDSTD/silver/covid_table"

# Carregue a tabela Delta Lake existente
delta_table = DeltaTable.forPath(spark, covid_table)

# Seu DataFrame com os novos dados
df = dfCovid

# Escreva o DataFrame na tabela Delta Lake existente
df.write.format("delta").mode("append").save(covid_table)

In [21]:
spark.sql(
    """
    SELECT *
    FROM silver.covid_table
    """
).toPandas()

SerialNumber       CountryName  TotalCases  TotalDeaths  TotalRecovered  \
0               1               USA         NaN          NaN     101322779.0   
1               2             India         NaN          NaN      44150289.0   
2               3            France         NaN          NaN      39264546.0   
3               4           Germany         NaN          NaN      37398100.0   
4               5            Brazil         NaN          NaN      35919372.0   
..            ...               ...         ...          ...             ...   
226           227  Diamond Princess       712.0         13.0           699.0   
227           228      Vatican City        29.0          NaN            29.0   
228           229    Western Sahara        10.0          1.0             9.0   
229           230        MS Zaandam         9.0          2.0             7.0   
230           231           Tokelau         5.0          NaN             NaN   

        TotalTest    Population  
0    1.159833e+09  3.348053e+08  
1    9.152658e+08  1.406632e+09  
2    2.714902e+08  6.558452e+07  
3    1.223324e+08  8.388360e+07  
4    6.377617e+07  2.153536e+08  
..            ...           ...  
226           NaN           NaN  
227           NaN  7.990000e+02  
228           NaN  6.261610e+05  
229           NaN           NaN  
230           NaN  1.378000e+03  

[231 rows x 7 columns]

In [8]:
#############################################################################################################################################################################
###################################### CONSUMO ALCOOL #######################################################################################################################
#############################################################################################################################################################################

In [10]:
#read hdfs file to dataframe
#
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("HDFSToDataFrame").getOrCreate()

hdfs_file_path = "hdfs://hdfs-nn:9000/projeto_EDSTD/bronze/ConsumoAlcool.csv"

dfConsumoAlcool = spark.read.option("delimiter",";").option("header","true").csv(hdfs_file_path)

#eliminar colunas
columns_to_drop = ['AUD_in_number','Death_due_to_AUD_deaths_by_population','Beer_consumption_per_capita_in_liter',
                   'Wine_Consuption_per_capita_in_liter','Spirit_Consuption_per_capita_in_liter']
dfConsumoAlcool = dfConsumoAlcool.drop(*columns_to_drop)

#rename colunas
dfConsumoAlcool = dfConsumoAlcool.withColumnRenamed("Country", "CountryName") \
                                 .withColumnRenamed("Death_per_annum", "DeathPerAnnum") \
                                 .withColumnRenamed("Alcohol_consumption_per_person_in_liter", "AlcoholConsumptionPerPersonInLiter") \
                                 .withColumnRenamed("Death_due_to_alcohol", "DeathDueToAlcohol")

# Converter StringType para IntegerType
dfConsumoAlcool = dfConsumoAlcool.withColumn("Population", col("Population").cast("integer"))
dfConsumoAlcool = dfConsumoAlcool.withColumn("DeathPerAnnum", col("DeathPerAnnum").cast("integer"))
dfConsumoAlcool = dfConsumoAlcool.withColumn("Year", col("Year").cast("integer"))
dfConsumoAlcool = dfConsumoAlcool.withColumn("AlcoholConsumptionPerPersonInLiter", col("AlcoholConsumptionPerPersonInLiter").cast("double"))
dfConsumoAlcool = dfConsumoAlcool.withColumn("DeathDueToAlcohol", col("DeathDueToAlcohol").cast("integer"))

dfConsumoAlcool.toPandas()

CountryName  Year  Population  DeathPerAnnum  \
0    Afghanistan  2000    20779957            430   
1        Albania  2000     3129246            511   
2        Algeria  2000    31042238            625   
3        Andorra  2000       65390             30   
4         Angola  2000    16395476           5664   
..           ...   ...         ...            ...   
930    Venezuela  2018    28887117           8199   
931      Vietnam  2018    95545959          51954   
932        Yemen  2018    28498683            827   
933       Zambia  2018    17351714           6710   
934     Zimbabwe  2018    14438812           5044   

     AlcoholConsumptionPerPersonInLiter  DeathDueToAlcohol  
0                                   NaN                 15  
1                                  6.57                 15  
2                                  0.58                 89  
3                                 13.28                  1  
4                                  2.76                125  
..                                  ...                ...  
930                                4.14                175  
931                                8.66                979  
932                                0.05                 79  
933                                6.54                132  
934                                4.67                 51  

[935 rows x 6 columns]

In [12]:
from delta import DeltaTable
from pyspark.sql import SparkSession

# Inicialize a sessão Spark
spark = SparkSession.builder.appName("Write to Delta Lake").getOrCreate()

# Caminho da tabela Delta no HDFS
consumoAlcool_table = "hdfs://hdfs-nn:9000/projeto_EDSTD/silver/consumoAlcool_table"

# Carregue a tabela Delta Lake existente
delta_table = DeltaTable.forPath(spark, consumoAlcool_table)

# Seu DataFrame com os novos dados
df = dfConsumoAlcool

# Escreva o DataFrame na tabela Delta Lake existente
df.write.format("delta").mode("append").save(consumoAlcool_table)

In [22]:
spark.sql(
    """
    SELECT *
    FROM silver.consumoAlcool_table
    """
).toPandas()

CountryName  Population  DeathPerAnnum  \
0    Afghanistan    29185511            492   
1        Albania     2948029            664   
2        Algeria    35977541            935   
3        Andorra       84454             38   
4         Angola    23356247          10471   
..           ...         ...            ...   
930    Venezuela    26432445           6904   
931      Vietnam    83832662          21892   
932        Yemen    20107416            717   
933       Zambia    11856244           4814   
934     Zimbabwe    12076697           4486   

     AlcoholConsumptionPerPersonInLiter  DeathDueToAlcohol  Year  
0                                  0.21                118  2010  
1                                  7.69                 17  2010  
2                                  0.65                 94  2010  
3                                 11.11                  1  2010  
4                                  8.16                158  2010  
..                                  ...                ...   ...  
930                                9.42                182  2005  
931                                2.63                651  2005  
932                                0.34                 51  2005  
933                                3.90                 87  2005  
934                                2.77                 41  2005  

[935 rows x 6 columns]

In [ ]:
#############################################################################################################################################################################
######################################## DESEMPREGO #########################################################################################################################
#############################################################################################################################################################################

In [19]:
#read hdfs file to dataframe
#
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("HDFSToDataFrame").getOrCreate()

hdfs_file_path = "hdfs://hdfs-nn:9000/projeto_EDSTD/bronze/Desemprego.csv"

dfDesemprego = spark.read.option("delimiter",";").option("header","true").csv(hdfs_file_path)

#eliminar colunas
columns_to_drop = ['1991','_c33']
dfDesemprego = dfDesemprego.drop(*columns_to_drop)

#rename colunas
dfDesemprego = dfDesemprego.withColumnRenamed("Country Name", "CountryName") \
                           .withColumnRenamed("Country Code", "CountryCode") 

# Unpivot - Columns to rows
from pyspark.sql.functions import col, expr

dfDesemprego = dfDesemprego.select(
        col("CountryName"),
        col("CountryCode"),
        expr("stack(30, '1992', `1992`, '1993', `1993`, '1994', `1994`, '1995', `1995`, '1996', `1996`, '1997', `1997`, '1998', `1998`, '1999', `1999`, '2000', `2000`, '2001', `2001`, '2002', `2002`, '2003', `2003`, '2004', `2004`, '2005', `2005`, '2006', `2006`, '2007', `2007`, '2008', `2008`, '2009', `2009`, '2010', `2010`, '2011', `2011`, '2012', `2012`, '2013', `2013`, '2014', `2014`, '2015', `2015`, '2016', `2016`, '2017', `2017`, '2018', `2018`, '2019', `2019`, '2020', `2020`, '2021', `2021`) as (Year,UnemploymentRate)")
)

# Converter StringType para IntegerType
dfDesemprego = dfDesemprego.withColumn("UnemploymentRate", col("UnemploymentRate").cast("Decimal(5, 2)"))
dfDesemprego = dfDesemprego.withColumn("Year", col("Year").cast("integer"))

dfDesemprego.toPandas()

CountryName CountryCode  Year UnemploymentRate
0     Africa Eastern and Southern         AFE  1992             7.84
1     Africa Eastern and Southern         AFE  1993             7.85
2     Africa Eastern and Southern         AFE  1994             7.84
3     Africa Eastern and Southern         AFE  1995             7.83
4     Africa Eastern and Southern         AFE  1996             7.84
...                           ...         ...   ...              ...
7045                     Zimbabwe         ZWE  2017             4.78
7046                     Zimbabwe         ZWE  2018             4.80
7047                     Zimbabwe         ZWE  2019             4.83
7048                     Zimbabwe         ZWE  2020             5.35
7049                     Zimbabwe         ZWE  2021             5.17

[7050 rows x 4 columns]

In [20]:
from delta import DeltaTable
from pyspark.sql import SparkSession

# Inicialize a sessão Spark
spark = SparkSession.builder.appName("Write to Delta Lake").getOrCreate()

# Caminho da tabela Delta no HDFS
desemprego_table = "hdfs://hdfs-nn:9000/projeto_EDSTD/silver/desemprego_table"

# Carregue a tabela Delta Lake existente
delta_table = DeltaTable.forPath(spark, desemprego_table)

# Seu DataFrame com os novos dados
df = dfDesemprego

# Escreva o DataFrame na tabela Delta Lake existente
df.write.format("delta").mode("append").save(desemprego_table)

In [23]:
spark.sql(
    """
    SELECT *
    FROM silver.desemprego_table
    """
).toPandas()

CountryName CountryCode UnemploymentRate  Year
0     Africa Eastern and Southern         AFE             7.96  2002
1                     Afghanistan         AFG            11.26  2002
2      Africa Western and Central         AFW             4.78  2002
3                          Angola         AGO             3.96  2002
4                         Albania         ALB            17.90  2002
...                           ...         ...              ...   ...
7045                        Samoa         WSM             2.38  1992
7046                        Yemen        Rep.             8.32  1992
7047                 South Africa         ZAF            29.98  1992
7048                       Zambia         ZMB            19.37  1992
7049                     Zimbabwe         ZWE             4.99  1992

[7050 rows x 4 columns]

In [ ]:
#############################################################################################################################################################################
################################## ESTATISTICAS POPULACIONAIS ###############################################################################################################
#############################################################################################################################################################################

In [2]:
#read hdfs file to dataframe
#
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

spark = SparkSession.builder.appName("HDFSToDataFrame").getOrCreate()

hdfs_file_path = "hdfs://hdfs-nn:9000/projeto_EDSTD/bronze/EstatisticasPopulacionais.csv"

dfEstatisticasPopulacionais = spark.read.option("delimiter",";").option("header","true").csv(hdfs_file_path)

#eliminar linhas
strings_a_excluir = [
    "A woman can apply for a passport in the same way as a man (1=yes",
    "A woman can be head of household in the same way as a man (1=yes",
    "A woman can choose where to live in the same way as a man (1=yes",
    "A woman can get a job in the same way as a man (1=yes",
    "A woman can obtain a judgment of divorce in the same way as a man (1=yes",
    "A woman can open a bank account in the same way as a man (1=yes",
    "A woman can register a business in the same way as a man (1=yes",
    "A woman can sign a contract in the same way as a man (1=yes",
    "A woman can travel outside her home in the same way as a man (1=yes",
    "A woman can travel outside the country in the same way as a man (1=yes",
    "A woman can work at night in the same way as a man (1=yes",
    "A woman can work in a job deemed dangerous in the same way as a man (1=yes",
    "A woman can work in an industrial job in the same way as a man (1=yes",
    "A woman has the same rights to remarry as a man (1=yes",
    "Coming up with emergency funds in 30 days: not possible (% age 15+)",
    "Coming up with emergency funds in 30 days: not possible, female (% age 15+)",
    "Coming up with emergency funds in 30 days: not possible, male (% age 15+)",
    "Coming up with emergency funds in 30 days: possible (% age 15+)",
    "Coming up with emergency funds in 30 days: possible and not difficult at all (% age 15+)",
    "Coming up with emergency funds in 30 days: possible and not difficult at all, female (% age 15+)",
    "Coming up with emergency funds in 30 days: possible and not difficult at all, male (% age 15+)",
    "Coming up with emergency funds in 30 days: possible and not difficult or somewhat difficult (% age 15+)",
    "Coming up with emergency funds in 30 days: possible and not difficult or somewhat difficult, female (% age 15+)",
    "Coming up with emergency funds in 30 days: possible and not difficult or somewhat difficult, male (% age 15+)",
    "Coming up with emergency funds in 30 days: possible and somewhat difficult (% age 15+)",
    "Coming up with emergency funds in 30 days: possible and somewhat difficult, female (% age 15+)",
    "Coming up with emergency funds in 30 days: possible and somewhat difficult, male (% age 15+)",
    "Coming up with emergency funds in 30 days: possible and very difficult (% age 15+)",
    "Coming up with emergency funds in 30 days: possible and very difficult, female (% age 15+)",
    "Coming up with emergency funds in 30 days: possible and very difficult, male (% age 15+)",
    "Coming up with emergency funds in 30 days: possible, female (% age 15+)",
    "Coming up with emergency funds in 30 days: possible, male (% age 15+)",
    "Coming up with emergency funds in 7 days: possible (% age 15+)",
    "Coming up with emergency funds in 7 days: possible and not difficult (% age 15+)",
    "Coming up with emergency funds in 7 days: possible and not difficult or somewhat difficult (% age 15+)",
    "Coming up with emergency funds in 7 days: possible and not difficult or somewhat difficult, female (% age 15+)",
    "Coming up with emergency funds in 7 days: possible and not difficult or somewhat difficult, male (% age 15+)",
    "Coming up with emergency funds in 7 days: possible and not difficult, female (% age 15+)",
    "Coming up with emergency funds in 7 days: possible and not difficult, male (% age 15+)",
    "Coming up with emergency funds in 7 days: possible and somewhat difficult (% age 15+)",
    "Coming up with emergency funds in 7 days: possible and somewhat difficult, female (% age 15+)",
    "Coming up with emergency funds in 7 days: possible and somewhat difficult, male (% age 15+)",
    "Coming up with emergency funds in 7 days: possible and very difficult (% age 15+)",
    "Coming up with emergency funds in 7 days: possible and very difficult, female (% age 15+)",
    "Coming up with emergency funds in 7 days: possible and very difficult, male (% age 15+)",
    "Coming up with emergency funds in 7 days: possible, female (% age 15+)",
    "Coming up with emergency funds in 7 days: possible, male (% age 15+)",
    "Female genital mutilation prevalence (%)",
    "Female genital mutilation prevalence (%): Q1 (lowest)",
    "Female genital mutilation prevalence (%): Q2",
    "Female genital mutilation prevalence (%): Q3",
    "Female genital mutilation prevalence (%): Q4",
    "Female genital mutilation prevalence (%): Q5 (highest)",
    "Female headed households (% of households with a female head)",
    "Female migrants (% of international migrant stock)",
    "Female primary school age children out-of-school (%)",
    "Female professional and technical workers (% of total)",
    "Female pupils below minimum reading proficiency at end of primary (%). Low GAML threshold",
    "Female share of employment in senior and middle management (%)",
    "Female share of graduates from Science, Technology, Engineering and Mathematics (STEM) programs, tertiary (%)",
    "Female share of graduates in Agriculture, Forestry, Fisheries and Veterinary programs, tertiary (%)",
    "Female share of graduates in Arts and Humanities programs, tertiary (%)",
    "Female share of graduates in Business, Administration and Law programs, tertiary (%)",
    "Female share of graduates in Education programs, tertiary (%)",
    "Female share of graduates in Engineering, Manufacturing and Construction programs, tertiary (%)",
    "Female share of graduates in Health and Welfare programs, tertiary (%)",
    "Female share of graduates in Information and Communication Technologies programs, tertiary (%)",
    "Female share of graduates in Natural Sciences, Mathematics and Statistics programs, tertiary (%)",
    "Female share of graduates in other fields than Science, Technology, Engineering and Mathematics programs, tertiary (%)",
    "Female share of graduates in Services programs, tertiary (%)",
    "Female share of graduates in Social Sciences, Journalism and Information programs, tertiary (%)",
    "Female share of graduates in unknown or unspecified fields, tertiary (%)",
    "Main cooking fuel: agricultural crop (% of households)",
    "Main cooking fuel: charcoal (% of households)",
    "Main cooking fuel: dung (% of households)",
    "Main cooking fuel: electricity  (% of households)",
    "Main cooking fuel: LPG/natural gas/biogas (% of households)",
    "Main cooking fuel: straw/shrubs/grass (% of households)",
    "Main cooking fuel: wood (% of households)",
    "Proportion of seats held by women in national parliaments (%)",
    "Proportion of time spent on unpaid domestic and care work, female (% of 24 hour day)",
    "Proportion of time spent on unpaid domestic and care work, male (% of 24 hour day)",
    "Proportion of women in ministerial level positions (%)",
    "Proportion of women subjected to physical and/or sexual violence in the last 12 months (% of ever-partnered women ages 15-49)",
    "Proportion of women subjected to physical and/or sexual violence in the last 12 months (modeled estimate, % of ever partnered women ages 15+)",
    "Proportion of women subjected to physical and/or sexual violence in the last 12 months (modeled estimate, % of ever partnered women ages 15-19)",
    "Proportion of women subjected to physical and/or sexual violence in the last 12 months (modeled estimate, % of ever partnered women ages 15-49)",
    "Proportion of women subjected to physical and/or sexual violence in the last 12 months (modeled estimate, % of ever partnered women ages 20-24)",
    "Proportion of women subjected to physical and/or sexual violence in the last 12 months (modeled estimate, % of ever partnered women ages 25-29)",
    "Proportion of women subjected to physical and/or sexual violence in the last 12 months (modeled estimate, % of ever partnered women ages 30-34)",
    "Proportion of women subjected to physical and/or sexual violence in the last 12 months (modeled estimate, % of ever partnered women ages 35-44)",
    "Proportion of women subjected to physical and/or sexual violence in the last 12 months (modeled estimate, % of ever partnered women ages 45-54)",
    "Proportion of women subjected to physical and/or sexual violence in the last 12 months (modeled estimate, % of ever partnered women ages 55-64)",
    "Proportion of women subjected to physical and/or sexual violence in the last 12 months (modeled estimate, % of ever partnered women ages 65+)",
    "Proportion of women who have ever experienced any form of sexual violence (% of women ages 15-49)",
    "Proportion of women who have ever experienced intimate partner violence (% of ever-married women ages 15-49)",
    "Proportion of women who have ever experienced intimate partner violence (modeled estimate, % of ever partnered women ages 15+)",
    "Proportion of women who have ever experienced intimate partner violence (modeled estimate, % of ever partnered women ages 15-19)",
    "Proportion of women who have ever experienced intimate partner violence (modeled estimate, % of ever partnered women ages 15-49)",
    "Proportion of women who have ever experienced intimate partner violence (modeled estimate, % of ever partnered women ages 20-24)",
    "Proportion of women who have ever experienced intimate partner violence (modeled estimate, % of ever partnered women ages 25-29)",
    "Proportion of women who have ever experienced intimate partner violence (modeled estimate, % of ever partnered women ages 30-34)",
    "Proportion of women who have ever experienced intimate partner violence (modeled estimate, % of ever partnered women ages 35-44)",
    "Proportion of women who have ever experienced intimate partner violence (modeled estimate, % of ever partnered women ages 45-54)",
    "Proportion of women who have ever experienced intimate partner violence (modeled estimate, % of ever partnered women ages 55-64)",
    "Proportion of women who have ever experienced intimate partner violence (modeled estimate, % of ever partnered women ages 65+)",
    "Proportion of women who have sought help to stop physical or sexual violence (% of ever-married women ages 15-49)",
    "Reason for not having a mobile money account: available mobile money products are too expensive (% age 15+)",
    "Reason for not having a mobile money account: available mobile money products are too expensive (% without an account, age 15+)",
    "Reason for not having a mobile money account: do not have their own mobile phone (% age 15+)",
    "Reason for not having a mobile money account: do not have their own mobile phone (% without an account, age 15+)",
    "Reason for not having a mobile money account: don't have enough money to use a mobile money account (% age 15+)",
    "Reason for not having a mobile money account: don't have enough money to use a mobile money account (% without an account, age 15+)",
    "Reason for not having a mobile money account: don't have the necessary documentation (% age 15+)",
    "Reason for not having a mobile money account: don't have the necessary documentation (% without an account, age 15+)",
    "Reason for not having a mobile money account: mobile money agents are too far away (% age 15+)",
    "Reason for not having a mobile money account: mobile money agents are too far away (% without an account, age 15+)",
    "Reason for not having a mobile money account: use an agent or someone else to make payments (% age 15+)",
    "Reason for not having a mobile money account: use an agent or someone else to make payments (% without an account, age 15+)",
    "Reason for not using their inactive account: bank or financial institution is too far away (% age 15+)",
    "Reason for not using their inactive account: bank or financial institution is too far away (% with an inactive account, age 15+)",
    "Reason for not using their inactive account: don't feel comfortable using an account by themselves (% with an inactive account, age 15+)",
    "Reason for not using their inactive account: don't feel comfortable using the account by themselves (% age 15+)",
    "Reason for not using their inactive account: don't have enough money to use an account (% age 15+)",
    "Reason for not using their inactive account: don't have enough money to use an account (% with an inactive account, age 15+)",
    "Reason for not using their inactive account: don't trust banks or financial institutions (% age 15+)",
    "Reason for not using their inactive account: don't trust banks or financial institutions (% with an inactive account, age 15+)",
    "Reason for not using their inactive account: no need for an account (% age 15+)",
    "Reason for not using their inactive account: no need for an account (% with an inactive account, age 15+)",
    "Received a public sector pension (% age 15+)",
    "Received a public sector pension, female (% age 15+)",
    "Received a public sector pension, male (% age 15+)",
    "Received digital payments (% age 15+)",
    "Received digital payments, female (% age 15+)",
    "Received digital payments, male (% age 15+)",
    "Received domestic remittances (% age 15+)",
    "Received domestic remittances, female (% age 15+)",
    "Received domestic remittances, male (% age 15+)",
    "Received government payments (% age 15+)",
    "Received government payments, female (% age 15+)",
    "Received government payments, male (% age 15+)",
    "Received government transfer (% age 15+)",
    "Received government transfer or pension (% age 15+)",
    "Received government transfer or pension, female (% age 15+)",
    "Received government transfer or pension, male (% age 15+)",
    "Received government transfer, female (% age 15+)",
    "Received government transfer, male (% age 15+)",
    "Received payments for the sale of agricultural products, livestock, or crops (% age 15+)",
    "Received payments for the sale of agricultural products, livestock, or crops, female (% age 15+)",
    "Received payments for the sale of agricultural products, livestock, or crops, male (% age 15+)",
    "Urban population (% of total population)",
    "Urban population, female (% of total)",
    "Urban population, male (% of total)",
    "Use a mobile money account two or more times a month (% age 15+)",
    "Use a mobile money account two or more times a month (% with a mobile money account, age 15+)",
    "Used a mobile phone or the internet to access an account(% age 15+)",
    "Used a mobile phone or the internet to access an account, female (% age 15+)",
    "Used a mobile phone or the internet to access an account, male (% age 15+)",
    "Used a mobile phone or the internet to buy something online (% age 15+)",
    "Used a mobile phone or the internet to buy something online, female (% age 15+)",
    "Used a mobile phone or the internet to buy something online, male (% age 15+)",
    "Used a mobile phone or the internet to pay bills (% age 15+)",
    "Used a mobile phone or the internet to pay bills, female (% age 15+)",
    "Used a mobile phone or the internet to pay bills, male (% age 15+)",
    "Used a mobile phone or the internet to send money (% age 15+)",
    "Used a mobile phone or the internet to send money, female (% age 15+)",
    "Used a mobile phone or the internet to send money, male (% age 15+)",
    "Vocational and Technical enrolment (% of total secondary enrolment), female",
    "Vocational and Technical enrolment (% of total secondary enrolment), total",
    "Vulnerable employment, female (% of female employment) (modeled ILO estimate)",
    "Vulnerable employment, male (% of male employment) (modeled ILO estimate)",
    "Women and girls who have private places to wash and change during menstrual period (% of women and girls ages 15-49 who had a menstrual period within the last year)",
    "Women and girls who have private places to wash and change during menstrual period, rural (% of women and girls ages 15-49 living in rural areas who had a menstrual period within the last year)",
    "Women and girls who have private places to wash and change during menstrual period, urban (% of women and girls ages 15-49 living in urban areas who had a menstrual period within the last year)",
    "Women and girls who participate in activities during menstrual period (% of women and girls ages 15-49 who had a menstrual period within the last year)",
    "Women and girls who participate in activities during menstrual period, rural (% of women and girls ages 15-49 living in rural areas who had a menstrual period within the last year)",
    "Women and girls who participate in activities during menstrual period, urban (% of women and girls ages 15-49 living in urban areas who had a menstrual period within the last year)",
    "Women and girls who use menstrual materials (% of women and girls ages 15-49 who had a menstrual period within the last year)",
    "Women and girls who use menstrual materials, rural (% of women and girls ages 15-49 living in rural areas who had a menstrual period within the last year)",
    "Women and girls who use menstrual materials, urban (% of women and girls ages 15-49 living in urban areas who had a menstrual period within the last year)",
    "Women Business and the Law Index Score (scale 1-100)",
    "Women making their own informed decisions regarding sexual relations, contraceptive use and reproductive health care  (% of women age 15-49)",
    "Women participating in decision of visits to family, relatives, friends (% of women age 15-49)",
    "Women participating in decision of what food to cook daily (% of women age 15-49)",
    "Women participating in making daily purchase decisions (% of women age 15-49)",
    "Women participating in making major household purchase decisions (% of women age 15-49)",
    "Women participating in none of the three decisions (own health care, major household purchases, and visiting family) (% of women age 15-49)",
    "Women participating in own health care decisions (% of women age 15-49)",
    "Women participating in the three decisions (own health care, major household purchases, and visiting family) (% of women age 15-49)",
    "Women who believe a husband is justified in beating his wife (any of five reasons) (%)",
    "Women who believe a husband is justified in beating his wife (any of five reasons) (%): Q1 (lowest)",
    "Women who believe a husband is justified in beating his wife (any of five reasons) (%): Q2",
    "Women who believe a husband is justified in beating his wife (any of five reasons) (%): Q3",
    "Women who believe a husband is justified in beating his wife (any of five reasons) (%): Q4",
    "Women who believe a husband is justified in beating his wife (any of five reasons) (%): Q5 (highest)",
    "Women who believe a husband is justified in beating his wife when she argues with him (%)",
    "Women who believe a husband is justified in beating his wife when she argues with him (%): Q1 (lowest)",
    "Women who believe a husband is justified in beating his wife when she argues with him (%): Q2",
    "Women who believe a husband is justified in beating his wife when she argues with him (%): Q3",
    "Women who believe a husband is justified in beating his wife when she argues with him (%): Q4",
    "Women who believe a husband is justified in beating his wife when she argues with him (%): Q5 (highest)",
    "Women who believe a husband is justified in beating his wife when she burns the food (%)",
    "Women who believe a husband is justified in beating his wife when she burns the food (%): Q1 (lowest)",
    "Women who believe a husband is justified in beating his wife when she burns the food (%): Q2",
    "Women who believe a husband is justified in beating his wife when she burns the food (%): Q3",
    "Women who believe a husband is justified in beating his wife when she burns the food (%): Q4",
    "Women who believe a husband is justified in beating his wife when she burns the food (%): Q5 (highest)",
    "Women who believe a husband is justified in beating his wife when she goes out without telling him (%)",
    "Women who believe a husband is justified in beating his wife when she goes out without telling him (%): Q1 (lowest)",
    "Women who believe a husband is justified in beating his wife when she goes out without telling him (%): Q2",
    "Women who believe a husband is justified in beating his wife when she goes out without telling him (%): Q3",
    "Women who believe a husband is justified in beating his wife when she goes out without telling him (%): Q4",
    "Women who believe a husband is justified in beating his wife when she goes out without telling him (%): Q5 (highest)",
    "Women who believe a husband is justified in beating his wife when she neglects the children (%)",
    "Women who believe a husband is justified in beating his wife when she neglects the children (%): Q1 (lowest)",
    "Women who believe a husband is justified in beating his wife when she neglects the children (%): Q2",
    "Women who believe a husband is justified in beating his wife when she neglects the children (%): Q3",
    "Women who believe a husband is justified in beating his wife when she neglects the children (%): Q4",
    "Women who believe a husband is justified in beating his wife when she neglects the children (%): Q5 (highest)",
    "Women who believe a husband is justified in beating his wife when she refuses sex with him (%)",
    "Women who believe a husband is justified in beating his wife when she refuses sex with him (%): Q1 (lowest)",
    "Women who believe a husband is justified in beating his wife when she refuses sex with him (%): Q2",
    "Women who believe a husband is justified in beating his wife when she refuses sex with him (%): Q3",
    "Women who believe a husband is justified in beating his wife when she refuses sex with him (%): Q4",
    "Women who believe a husband is justified in beating his wife when she refuses sex with him (%): Q5 (highest)",
    "Women who believe a wife is justified refusing sex with her husband for all of the reasons (%)",
    "Women who believe a wife is justified refusing sex with her husband for all of the reasons (%): Q1 (lowest)",
    "Women who believe a wife is justified refusing sex with her husband for all of the reasons (%): Q2",
    "Women who believe a wife is justified refusing sex with her husband for all of the reasons (%): Q3",
    "Women who believe a wife is justified refusing sex with her husband for all of the reasons (%): Q4",
    "Women who believe a wife is justified refusing sex with her husband for all of the reasons (%): Q5 (highest)",
    "Women who believe a wife is justified refusing sex with her husband for none of the reasons (%)",
    "Women who believe a wife is justified refusing sex with her husband for none of the reasons (%): Q1 (lowest)",
    "Women who believe a wife is justified refusing sex with her husband for none of the reasons (%): Q2",
    "Women who believe a wife is justified refusing sex with her husband for none of the reasons (%): Q3",
    "Women who believe a wife is justified refusing sex with her husband for none of the reasons (%): Q4",
    "Women who believe a wife is justified refusing sex with her husband for none of the reasons (%): Q5 (highest)",
    "Women who believe a wife is justified refusing sex with her husband if she has recently given birth (%)",
    "Women who believe a wife is justified refusing sex with her husband if she has recently given birth (%): Q1 (lowest)",
    "Women who believe a wife is justified refusing sex with her husband if she has recently given birth (%): Q2",
    "Women who believe a wife is justified refusing sex with her husband if she has recently given birth (%): Q3",
    "Women who believe a wife is justified refusing sex with her husband if she has recently given birth (%): Q4",
    "Women who believe a wife is justified refusing sex with her husband if she has recently given birth (%): Q5 (highest)",
    "Women who believe a wife is justified refusing sex with her husband if she is tired or not in the mood (%)",
    "Women who believe a wife is justified refusing sex with her husband if she is tired or not in the mood (%): Q1 (lowest)",
    "Women who believe a wife is justified refusing sex with her husband if she is tired or not in the mood (%): Q2",
    "Women who believe a wife is justified refusing sex with her husband if she is tired or not in the mood (%): Q3",
    "Women who believe a wife is justified refusing sex with her husband if she is tired or not in the mood (%): Q4",
    "Women who believe a wife is justified refusing sex with her husband if she is tired or not in the mood (%): Q5 (highest)",
    "Women who believe a wife is justified refusing sex with her husband if she knows he has sex with other women (%)",
    "Women who believe a wife is justified refusing sex with her husband if she knows he has sex with other women (%): Q1 (lowest)",
    "Women who believe a wife is justified refusing sex with her husband if she knows he has sex with other women (%): Q2",
    "Women who believe a wife is justified refusing sex with her husband if she knows he has sex with other women (%): Q3",
    "Women who believe a wife is justified refusing sex with her husband if she knows he has sex with other women (%): Q4",
    "Women who believe a wife is justified refusing sex with her husband if she knows he has sex with other women (%): Q5 (highest)",
    "Women who believe a wife is justified refusing sex with her husband if she knows he has sexually transmitted disease (%)",
    "Women who believe a wife is justified refusing sex with her husband if she knows he has sexually transmitted disease (%): Q1 (lowest)",
    "Women who believe a wife is justified refusing sex with her husband if she knows he has sexually transmitted disease (%): Q2",
    "Women who believe a wife is justified refusing sex with her husband if she knows he has sexually transmitted disease (%): Q3",
    "Women who believe a wife is justified refusing sex with her husband if she knows he has sexually transmitted disease (%): Q4",
    "Women who believe a wife is justified refusing sex with her husband if she knows he has sexually transmitted disease (%): Q5 (highest)",
    "Women who believe religion requires female genital mutilation (% of women ages 15-49 who have heard about FGM)",
    "Women who believe that female genital mutilation should not be continued (% of women ages 15-49 who have heard about FGM)",
    "Women who do not own a house (% of women age 15-49)",
    "Women who do not own a house (% of women age 15-49): Q1 (lowest)",
    "Women who do not own a house (% of women age 15-49): Q2",
    "Women who do not own a house (% of women age 15-49): Q3",
    "Women who do not own a house (% of women age 15-49): Q4",
    "Women who do not own a house (% of women age 15-49): Q5 (highest)",
    "Women who do not own land (% of women age 15-49)",
    "Women who do not own land (% of women age 15-49): Q1 (lowest)",
    "Women who do not own land (% of women age 15-49): Q2",
    "Women who do not own land (% of women age 15-49): Q3",
    "Women who do not own land (% of women age 15-49): Q4",
    "Women who do not own land (% of women age 15-49): Q5 (highest)",
    "Women who experienced first sexual violence before age 15 (% of women ages 15-49)",
    "Women who experienced first sexual violence before age 18 (% of women ages 15-49)",
    "Women who experienced first sexual violence before age 22 (% of women ages 15-49)",
    "Women who have ever experienced  sexual violence committed by their husband/partner  (% of ever-married women ages 15-49)",
    "Women who have ever experienced emotional violence committed by their husband/partner  (% of ever-married women ages 15-49)",
    "Women who have ever experienced physical violence committed by their husband/partner (% of ever-married women ages 15-49)",
    "Women who have experienced emotional violence committed by their husband/partner in the 12 months (% of ever-married women ages 15-49)",
    "Women who have experienced physical violence committed by their husband/partner in the 12 months (% of ever-married women ages 15-49)",
    "Women who have experienced sexual violence committed by their husband/partner in the 12 months (% of ever-married women ages 15-49)",
    "Women who have not experienced spousal physical or sexual violence (% of currently married women age 15-49 who have been married only once)",
    "Women who initiated sexual intercourse before age 15 (% of women ages 15-19)",
    "Women who initiated sexual intercourse by age 15 (% of women ages 20-24)",
    "Women who never sought help to stop violence, and never told anyone  (% of ever-married women ages 15-49 who have ever experienced any physical or sexual violence)",
    "Women who never sought help to stop violence, but told someone (% of ever-married women ages 15-49 who have ever experienced any physical or sexual violence)",
    "Women who own a house alone (% of women age 15-49)",
    "Women who own a house alone (% of women age 15-49): Q1 (lowest)",
    "Women who own a house alone (% of women age 15-49): Q2",
    "Women who own a house alone (% of women age 15-49): Q3",
    "Women who own a house alone (% of women age 15-49): Q4",
    "Women who own a house alone (% of women age 15-49): Q5 (highest)",
    "Women who own a house both alone and jointly (% of women age 15-49)",
    "Women who own a house both alone and jointly (% of women age 15-49): Q1 (lowest)",
    "Women who own a house both alone and jointly (% of women age 15-49): Q2",
    "Women who own a house both alone and jointly (% of women age 15-49): Q3",
    "Women who own a house both alone and jointly (% of women age 15-49): Q4",
    "Women who own a house both alone and jointly (% of women age 15-49): Q5 (highest)",
    "Women who own a house jointly (% of women age 15-49)",
    "Women who own a house jointly (% of women age 15-49): Q1 (lowest)",
    "Women who own a house jointly (% of women age 15-49): Q2",
    "Women who own a house jointly (% of women age 15-49): Q3",
    "Women who own a house jointly (% of women age 15-49): Q4",
    "Women who own a house jointly (% of women age 15-49): Q5 (highest)",
    "Women who own land alone (% of women age 15-49)",
    "Women who own land alone (% of women age 15-49): Q1 (lowest)",
    "Women who own land alone (% of women age 15-49): Q2",
    "Women who own land alone (% of women age 15-49): Q3",
    "Women who own land alone (% of women age 15-49): Q4",
    "Women who own land alone (% of women age 15-49): Q5 (highest)",
    "Women who own land both alone and jointly (% of women age 15-49)",
    "Women who own land both alone and jointly (% of women age 15-49): Q1 (lowest)",
    "Women who own land both alone and jointly (% of women age 15-49): Q2",
    "Women who own land both alone and jointly (% of women age 15-49): Q3",
    "Women who own land both alone and jointly (% of women age 15-49): Q4",
    "Women who own land both alone and jointly (% of women age 15-49): Q5 (highest)",
    "Women who own land jointly (% of women age 15-49)",
    "Women who own land jointly (% of women age 15-49): Q1 (lowest)",
    "Women who own land jointly (% of women age 15-49): Q2",
    "Women who own land jointly (% of women age 15-49): Q3",
    "Women who own land jointly (% of women age 15-49): Q4",
    "Women who own land jointly (% of women age 15-49): Q5 (highest)",
    "Women who were first married by age 15 (% of women ages 20-24)",
    "Women who were first married by age 18 (% of women ages 20-24)",
    "Women who who have experienced injuries resulting from spousal violence (% of ever-married women ages 15-49 who have ever experienced any physical or sexual violence)",
    "Women whose first experience of spousal physical or sexual violence was before marriage  (% of currently married women age 15-49 who have been married only once)",
    "Women whose first experience of spousal physical or sexual violence was within five years of marriage  (% of currently married women age 15-49 who have been married only once)",
    "Women whose first experience of spousal physical or sexual violence was within ten years of marriage  (% of currently married women age 15-49 who have been married only once)",
    "Women whose first experience of spousal physical or sexual violence was within two years of marriage  (% of currently married women age 15-49 who have been married only once)",
    "Women whose husband or partner has ever demonstrated controlling behaviors (% of ever-married women ages 15-49)",
    "Women whose husband or partner has never demonstrated controlling behaviors (% of ever-married women ages 15-49)",
    "Women, Business and the Law: Assets Indicator Score (scale 1-100)",
    "Women, Business and the Law: Entrepreneurship Indicator Score (scale 1-100)",
    "Women, Business and the Law: Marriage Indicator Score (scale 1-100)",
    "Women, Business and the Law: Mobility Indicator Score (scale 1-100)",
    "Women, Business and the Law: Parenthood Indicator Score (scale 1-100)",
    "Women, Business and the Law: Pay Indicator Score (scale 1-100)",
    "Women, Business and the Law: Pension Indicator Score (scale 1-100)",
    "Women, Business and the Law: Workplace Indicator Score (scale 1-100)",
    "Access to anti-retroviral drugs, female (%)",
    "Access to anti-retroviral drugs, male (%)",
    "Account (% age 15+)",
    "Account, female (% age 15+)",
    "Account, male (% age 15+)",
    "Age at first marriage, female",
    "Age at first marriage, male",
    "Age dependency ratio (% of working-age population)",
    "Age population, age 0, female, interpolated",
    "Age population, age 0, male, interpolated",
    "Age population, age 01, female, interpolated",
    "Age population, age 01, male, interpolated",
    "Age population, age 02, female, interpolated",
    "Age population, age 02, male, interpolated",
    "Age population, age 03, female, interpolated",
    "Age population, age 03, male, interpolated",
    "Age population, age 04, female, interpolated",
    "Age population, age 04, male, interpolated",
    "Age population, age 05, female, interpolated",
    "Age population, age 05, male, interpolated",
    "Borrowed any money (% age 15+)",
    "Borrowed any money from a formal financial institution or using a mobile money account (% age 15+)",
    "Borrowed any money from a formal financial institution or using a mobile money account, female (% age 15+)",
    "Borrowed any money from a formal financial institution or using a mobile money account, male (% age 15+)",
    "Borrowed any money, female (% age 15+)",
    "Borrowed any money, male (% age 15+)",
    "Borrowed for education or school fees (% age 15+)",
    "Borrowed for education or school fees, female (% age 15+)",
    "Borrowed for education or school fees, male (% age 15+)",
    "Borrowed for health or medical purposes (% age 15+)",
    "Borrowed for health or medical purposes, female (% age 15+)",
    "Borrowed for health or medical purposes, male (% age 15+)",
    "Borrowed from a formal financial institution (% age 15+)",
    "Borrowed from a formal financial institution, female (% age 15+)",
    "Borrowed from a formal financial institution, male (% age 15+)",
    "Borrowed from a savings club (% age 15+)",
    "Borrowed from a savings club, female (% age 15+)",
    "Borrowed from a savings club, male (% age 15+)",
    "Borrowed from a store by buying on credit (% age 15+)",
    "Borrowed from a store by buying on credit, female (% age 15+)",
    "Borrowed from a store by buying on credit, male (% age 15+)",
    "Borrowed from family or friends (% age 15+)",
    "Borrowed from family or friends, female (% age 15+)",
    "Borrowed from family or friends, male (% age 15+)",
    "Borrowed to start, operate, or expand a farm or business (% age 15+)",
    "Borrowed to start, operate, or expand a farm or business, female (% age 15+)",
    "Borrowed to start, operate, or expand a farm or business, male (% age 15+)",
    "Cost of business start-up procedures (% of GNI per capita)",
    "Cost of business start-up procedures, female (% of GNI per capita)",
    "Cost of business start-up procedures, male (% of GNI per capita)",
    "Criminal penalties or civil remedies exist for sexual harassment in employment (1=yes",
    "Decision maker about a woman's own health care: mainly husband (% of women age 15-49)",
    "Decision maker about a woman's own health care: mainly wife (% of women age 15-49)",
    "Decision maker about a woman's own health care: other (% of women age 15-49)",
    "Decision maker about a woman's own health care: someone else (% of women age 15-49)",
    "Decision maker about a woman's own health care: wife and husband jointly (% of women age 15-49)",
    "Decision maker about a woman's visits to her family or relatives: mainly husband (% of women age 15-49)",
    "Decision maker about a woman's visits to her family or relatives: mainly wife (% of women age 15-49)",
    "Decision maker about a woman's visits to her family or relatives: other (% of women age 15-49)",
    "Decision maker about a woman's visits to her family or relatives: someone else (% of women age 15-49)",
    "Decision maker about major household purchases: mainly husband (% of women age 15-49)",
    "Decision maker about major household purchases: mainly wife (% of women age 15-49)",
    "Decision maker about major household purchases: other (% of women age 15-49)",
    "Decision maker about major household purchases: someone else (% of women age 15-49)",
    "Decision maker about major household purchases: wife and husband jointly (% of women age 15-49)",
    "Decision maker about Visits to her family or relatives: wife and husband jointly (% of women age 15-49)",
    "Demand for family planning satisfied by any methods (% of married women with demand for family planning)",
    "Demand for family planning satisfied by modern methods (% of married women with demand for family planning)",
    "Dismissal of pregnant workers is prohibited (1=yes",
    "Fertility planning status: mistimed pregnancy (% of births)",
    "Fertility planning status: planned pregnancy (% of births)",
    "Fertility planning status: unwanted pregnancy (% of births)",
    "First financial institution account ever was opened to receive a wage payment (% age 15+)",
    "First financial institution account ever was opened to receive a wage payment or money from the government (% age 15+)",
    "First financial institution ever account was opened to receive money from the government (% age 15+)",
    "Fraction of Children Under 5 Not Stunted",
    "Fraction of Children Under 5 Not Stunted, Female",
    "Fraction of Children Under 5 Not Stunted, Male",
    "Harmonized Test Scores",
    "Harmonized Test Scores, Female",
    "Harmonized Test Scores, Male",
    "Has an inactive account (% age 15+)",
    "Has an inactive account, female (% age 15+)",
    "Has an inactive account, male (% age 15+)",
    "Households with water 30 minutes or longer away round trip (%)",
    "Households with water less than 30 minutes away round trip (%)",
    "Households with water on the premises (%)",
    "Learning poverty: Share of Children at the End-of-Primary age below minimum reading proficiency adjusted by Out-of-School Children (%)",
    "Learning poverty: Share of Female Children at the End-of-Primary age below minimum reading proficiency adjusted by Out-of-School Children (%)",
    "Learning poverty: Share of Male Children at the End-of-Primary age below minimum reading proficiency adjusted by Out-of-School Children (%)",
    "Learning-Adjusted Years of School",
    "Learning-Adjusted Years of School, Female",
    "Learning-Adjusted Years of School, Male",
    "Length of paid maternity leave (calendar days)",
    "Length of paid parental leave for father (calendar days)",
    "Length of paid parental leave for mother (calendar days)",
    "Length of paid paternity leave (calendar days)",
    "Length of paid shared parental leave (calendar days)",
    "Literacy rate, adult female (% of females ages 15 and above)",
    "Literacy rate, adult male (% of males ages 15 and above)",
    "Literacy rate, adult total (% of people ages 15 and above)",
    "Literacy rate, youth (ages 15-24), gender parity index (GPI)",
    "Literacy rate, youth female (% of females ages 15-24)",
    "Literacy rate, youth male (% of males ages 15-24)",
    "Literacy rate, youth total (% of people ages 15-24)",
    "Location of cooking: inside the house (% of households)",
    "Location of cooking: other places (% of households)",
    "Location of cooking: outdoors (% of households)",
    "Location of cooking: separate building (% of households)",
    "Lower secondary completion rate, female (% of relevant age group)",
    "Lower secondary completion rate, male (% of relevant age group)",
    "Lower secondary completion rate, total (% of relevant age group)",
    "Made a digital in-store merchant payment: using a mobile phone (% age 15+)",
    "Made a digital in-store merchant payment: using a mobile phone, female (% age 15+)",
    "Made a digital in-store merchant payment: using a mobile phone, male (% age 15+)",
    "Made a digital merchant payment (% age 15+)",
    "Made a digital merchant payment, female (% age 15+)",
    "Made a digital merchant payment, male (% age 15+)",
    "Made a digital payment (% age 15+)",
    "Made a digital payment, female (% age 15+)",
    "Made a digital payment, male (% age 15+)",
    "Made a utility payment (% age 15+)",
    "Made a utility payment, female (% age 15+)",
    "Made a utility payment, male (% age 15+)",
    "Made or received a digital payment (% age 15+)",
    "Made or received a digital payment, female (% age 15+)",
    "Made or received a digital payment, male (% age 15+)",
    "Male and female surviving spouses have equal rights to inherit assets (1=yes)",
    "Mandatory retirement age, female",
    "Mandatory retirement age, male",
    "Men and women have equal ownership rights to immovable property (1=yes)",
    "Men who believe religion requires female genital mutilation  (% of men who have heard about FGM)",
    "Men who believe that female genital mutilation should not be continued (% of men who have heard about FGM)",
    "Men who do not own a house (% of men)",
    "Men who do not own a house (% of men): Q1 (lowest)",
    "Men who do not own a house (% of men): Q2",
    "Men who do not own a house (% of men): Q3",
    "Men who do not own a house (% of men): Q4",
    "Men who do not own a house (% of men): Q5 (highest)",
    "Men who do not own land (% of men)",
    "Men who do not own land (% of men): Q1 (lowest)",
    "Men who do not own land (% of men): Q2",
    "Men who do not own land (% of men): Q3",
    "Men who do not own land (% of men): Q4",
    "Men who do not own land (% of men): Q5 (highest)",
    "Men who initiated sexual intercourse before age 15 (% of men ages 15-19)",
    "Men who initiated sexual intercourse by age 15 (% of men ages 20-24)",
    "Men who own a house alone (% of men)",
    "Men who own a house alone (% of men): Q1 (lowest)",
    "Men who own a house alone (% of men): Q2",
    "Men who own a house alone (% of men): Q3",
    "Men who own a house alone (% of men): Q4",
    "Men who own a house alone (% of men): Q5 (highest)",
    "Men who own a house both alone and jointly (% of men)",
    "Men who own a house both alone and jointly (% of men): Q1 (lowest)",
    "Men who own a house both alone and jointly (% of men): Q2",
    "Men who own a house both alone and jointly (% of men): Q3",
    "Men who own a house both alone and jointly (% of men): Q4",
    "Men who own a house both alone and jointly (% of men): Q5 (highest)",
    "Men who own a house jointly (% of men)",
    "Men who own a house jointly (% of men): Q1 (lowest)",
    "Men who own a house jointly (% of men): Q2",
    "Men who own a house jointly (% of men): Q3",
    "Men who own a house jointly (% of men): Q4",
    "Men who own a house jointly (% of men): Q5 (highest)",
    "Men who own land alone (% of men)",
    "Men who own land alone (% of men): Q1 (lowest)",
    "Men who own land alone (% of men): Q2",
    "Men who own land alone (% of men): Q3",
    "Men who own land alone (% of men): Q4",
    "Men who own land alone (% of men): Q5 (highest)",
    "Men who own land both alone and jointly (% of men)",
    "Men who own land both alone and jointly (% of men): Q1 (lowest)",
    "Men who own land both alone and jointly (% of men): Q2",
    "Men who own land both alone and jointly (% of men): Q3",
    "Men who own land both alone and jointly (% of men): Q4",
    "Men who own land both alone and jointly (% of men): Q5 (highest)",
    "Men who own land jointly (% of men)",
    "Men who own land jointly (% of men): Q1 (lowest)",
    "Men who own land jointly (% of men): Q2",
    "Men who own land jointly (% of men): Q3",
    "Men who own land jointly (% of men): Q4",
    "Men who own land jointly (% of men): Q5 (highest)",
    "Mobile money account (% age 15+)",
    "Youth illiterate population, 15-24 years, % female",
    "Youth illiterate population, 15-24 years, both sexes (number)",
    "Youth illiterate population, 15-24 years, female (number)",
    "Youth illiterate population, 15-24 years, male (number)"
]

# Elimine as linhas em que a coluna 'IndicatorName' contém qualquer uma das strings a excluir
dfEstatisticasPopulacionais = dfEstatisticasPopulacionais.filter(~col("Indicator Name").isin(strings_a_excluir))

dfEstatisticasPopulacionais.toPandas()


Country Name Country Code  \
0       Africa Eastern and Southern          AFE   
1       Africa Eastern and Southern          AFE   
2       Africa Eastern and Southern          AFE   
3       Africa Eastern and Southern          AFE   
4       Africa Eastern and Southern          AFE   
...                             ...          ...   
175508                     Zimbabwe          ZWE   
175509                     Zimbabwe          ZWE   
175510                     Zimbabwe          ZWE   
175511                     Zimbabwe          ZWE   
175512                     Zimbabwe          ZWE   

                                           Indicator Name  Indicator Code  \
0       Adjusted net enrollment rate, primary (% of pr...     SE.PRM.TENR   
1       Adjusted net enrollment rate, primary, female ...  SE.PRM.TENR.FE   
2       Adjusted net enrollment rate, primary, male (%...  SE.PRM.TENR.MA   
3       Adolescent fertility rate (births per 1,000 wo...     SP.ADO.TFRT   
4       Antiretroviral therapy coverage for PMTCT (% o...  SH.HIV.PMTC.ZS   
...                                                   ...             ...   
175508  Worried about not having enough money for old ...     fin44a2.d.1   
175509  Worried about not having enough money for old ...     fin44a2.d.2   
175510  Worried about not having enough money for old ...       fin44a1.d   
175511  Worried about not having enough money for old ...     fin44a1.d.1   
175512  Worried about not having enough money for old ...     fin44a1.d.2   

                    1960              1961              1962  \
0                   None              None              None   
1                   None              None              None   
2                   None              None              None   
3       140.180525679236  140.810248007317  141.664168114272   
4                   None              None              None   
...                  ...               ...               ...   
175508              None              None              None   
175509              None              None              None   
175510              None              None              None   
175511              None              None              None   
175512              None              None              None   

                    1963              1964              1965  ...  \
0                   None              None              None  ...   
1                   None              None              None  ...   
2                   None              None              None  ...   
3       142.324951230209  143.456932955638  144.331149937632  ...   
4                   None              None              None  ...   
...                  ...               ...               ...  ...   
175508              None              None              None  ...   
175509              None              None              None  ...   
175510              None              None              None  ...   
175511              None              None              None  ...   
175512              None              None              None  ...   

                    2014              2015              2016  \
0                   None              None              None   
1                   None              None              None   
2                   None              None              None   
3       107.697714993556   105.50183295561  103.222825498629   
4       85.3447535612551  85.9896753457356   85.883722323722   
...                  ...               ...               ...   
175508              None              None              None   
175509              None              None              None   
175510              None              None              None   
175511              None              None              None   
175512              None              None              None   

                    2017              2018              2019  \
0                   None              None

In [3]:
#eliminar colunas
columns_to_drop = ['1960', '1961', '1962', '1963', '1964', '1965', '1966', '1967', '1968', '1969', '1970', '1971', '1972', 
                   '1973', '1974', '1975', '1976', '1977', '1978', '1979', '1980', '1981', '1982', '1983', '1984', '1985', 
                   '1986', '1987', '1988', '1989', '1990', '1991','_c67']
dfEstatisticasPopulacionais = dfEstatisticasPopulacionais.drop(*columns_to_drop)

dfEstatisticasPopulacionais.toPandas()

Country Name Country Code  \
0       Africa Eastern and Southern          AFE   
1       Africa Eastern and Southern          AFE   
2       Africa Eastern and Southern          AFE   
3       Africa Eastern and Southern          AFE   
4       Africa Eastern and Southern          AFE   
...                             ...          ...   
175508                     Zimbabwe          ZWE   
175509                     Zimbabwe          ZWE   
175510                     Zimbabwe          ZWE   
175511                     Zimbabwe          ZWE   
175512                     Zimbabwe          ZWE   

                                           Indicator Name  Indicator Code  \
0       Adjusted net enrollment rate, primary (% of pr...     SE.PRM.TENR   
1       Adjusted net enrollment rate, primary, female ...  SE.PRM.TENR.FE   
2       Adjusted net enrollment rate, primary, male (%...  SE.PRM.TENR.MA   
3       Adolescent fertility rate (births per 1,000 wo...     SP.ADO.TFRT   
4       Antiretroviral therapy coverage for PMTCT (% o...  SH.HIV.PMTC.ZS   
...                                                   ...             ...   
175508  Worried about not having enough money for old ...     fin44a2.d.1   
175509  Worried about not having enough money for old ...     fin44a2.d.2   
175510  Worried about not having enough money for old ...       fin44a1.d   
175511  Worried about not having enough money for old ...     fin44a1.d.1   
175512  Worried about not having enough money for old ...     fin44a1.d.2   

                    1992              1993              1994  \
0                   None              None              None   
1                   None              None              None   
2                   None              None              None   
3       128.650226745135  129.800646275656  128.792351238038   
4                   None              None              None   
...                  ...               ...               ...   
175508              None              None              None   
175509              None              None              None   
175510              None              None              None   
175511              None              None              None   
175512              None              None              None   

                    1995              1996             1997  ...  \
0                   None              None             None  ...   
1                   None              None             None  ...   
2                   None              None             None  ...   
3       127.306550787239  128.239448076508  125.26199777544  ...   
4                   None              None             None  ...   
...                  ...               ...              ...  ...   
175508              None              None             None  ...   
175509              None              None             None  ...   
175510              None              None             None  ...   
175511              None              None             None  ...   
175512              None              None             None  ...   

                    2013              2014              2015  \
0                   None              None              None   
1                   None              None              None   
2                   None              None              None   
3       108.092795855758  107.697714993556   105.50183295561   
4       83.4099015088248  85.3447535612551  85.9896753457356   
...                  ...               ...               ...   
175508              None              None              None   
175509              None              None              None   
175510              None              None              None   
175511              None              None              None   
175512              None              None              None   

                    2016              2017              2018  \
0                   None              None            

In [5]:
#rename colunas
dfEstatisticasPopulacionais = dfEstatisticasPopulacionais.withColumnRenamed("Country Name", "CountryName") \
                                                         .withColumnRenamed("Country Code", "CountryCode") \
                                                         .withColumnRenamed("Indicator Name", "IndicatorName") \
                                                         .withColumnRenamed("Indicator Code", "IndicatorCode") 
dfEstatisticasPopulacionais.toPandas()

CountryName CountryCode  \
0       Africa Eastern and Southern         AFE   
1       Africa Eastern and Southern         AFE   
2       Africa Eastern and Southern         AFE   
3       Africa Eastern and Southern         AFE   
4       Africa Eastern and Southern         AFE   
...                             ...         ...   
175508                     Zimbabwe         ZWE   
175509                     Zimbabwe         ZWE   
175510                     Zimbabwe         ZWE   
175511                     Zimbabwe         ZWE   
175512                     Zimbabwe         ZWE   

                                            IndicatorName   IndicatorCode  \
0       Adjusted net enrollment rate, primary (% of pr...     SE.PRM.TENR   
1       Adjusted net enrollment rate, primary, female ...  SE.PRM.TENR.FE   
2       Adjusted net enrollment rate, primary, male (%...  SE.PRM.TENR.MA   
3       Adolescent fertility rate (births per 1,000 wo...     SP.ADO.TFRT   
4       Antiretroviral therapy coverage for PMTCT (% o...  SH.HIV.PMTC.ZS   
...                                                   ...             ...   
175508  Worried about not having enough money for old ...     fin44a2.d.1   
175509  Worried about not having enough money for old ...     fin44a2.d.2   
175510  Worried about not having enough money for old ...       fin44a1.d   
175511  Worried about not having enough money for old ...     fin44a1.d.1   
175512  Worried about not having enough money for old ...     fin44a1.d.2   

                    1992              1993              1994  \
0                   None              None              None   
1                   None              None              None   
2                   None              None              None   
3       128.650226745135  129.800646275656  128.792351238038   
4                   None              None              None   
...                  ...               ...               ...   
175508              None              None              None   
175509              None              None              None   
175510              None              None              None   
175511              None              None              None   
175512              None              None              None   

                    1995              1996             1997  ...  \
0                   None              None             None  ...   
1                   None              None             None  ...   
2                   None              None             None  ...   
3       127.306550787239  128.239448076508  125.26199777544  ...   
4                   None              None             None  ...   
...                  ...               ...              ...  ...   
175508              None              None             None  ...   
175509              None              None             None  ...   
175510              None              None             None  ...   
175511              None              None             None  ...   
175512              None              None             None  ...   

                    2013              2014              2015  \
0                   None              None              None   
1                   None              None              None   
2                   None              None              None   
3       108.092795855758  107.697714993556   105.50183295561   
4       83.4099015088248  85.3447535612551  85.9896753457356   
...                  ...               ...               ...   
175508              None              None              None   
175509              None              None              None   
175510              None              None              None   
175511              None              None              None   
175512              None              None              None   

                    2016              2017              2018  \
0                   None              None              None   
1  

In [6]:
# Unpivot - Columns to rows
from pyspark.sql.functions import col, expr

dfEstatisticasPopulacionais = dfEstatisticasPopulacionais.select(
        col("CountryName"),
        col("CountryCode"),
        col("IndicatorName"),
        col("IndicatorCode"),
        expr("stack(31, '1992', `1992`, '1993', `1993`, '1994', `1994`, '1995', `1995`, '1996', `1996`, '1997', `1997`, '1998', `1998`, '1999', `1999`, '2000', `2000`, '2001', `2001`, '2002', `2002`, '2003', `2003`, '2004', `2004`, '2005', `2005`, '2006', `2006`, '2007', `2007`, '2008', `2008`, '2009', `2009`, '2010', `2010`, '2011', `2011`, '2012', `2012`, '2013', `2013`, '2014', `2014`, '2015', `2015`, '2016', `2016`, '2017', `2017`, '2018', `2018`, '2019', `2019`, '2020', `2020`, '2021', `2021`, '2022', `2022`) as (Year,IndicatorValue)")
).where("IndicatorValue is not null")

# Converter StringType para IntegerType
dfEstatisticasPopulacionais = dfEstatisticasPopulacionais.withColumn("IndicatorValue", col("IndicatorValue").cast("double"))
dfEstatisticasPopulacionais = dfEstatisticasPopulacionais.withColumn("Year", col("Year").cast("integer"))

dfEstatisticasPopulacionais.toPandas()

CountryName CountryCode  \
0        Africa Eastern and Southern         AFE   
1        Africa Eastern and Southern         AFE   
2        Africa Eastern and Southern         AFE   
3        Africa Eastern and Southern         AFE   
4        Africa Eastern and Southern         AFE   
...                              ...         ...   
1788377                     Zimbabwe         ZWE   
1788378                     Zimbabwe         ZWE   
1788379                     Zimbabwe         ZWE   
1788380                     Zimbabwe         ZWE   
1788381                     Zimbabwe         ZWE   

                                             IndicatorName IndicatorCode  \
0        Adolescent fertility rate (births per 1,000 wo...   SP.ADO.TFRT   
1        Adolescent fertility rate (births per 1,000 wo...   SP.ADO.TFRT   
2        Adolescent fertility rate (births per 1,000 wo...   SP.ADO.TFRT   
3        Adolescent fertility rate (births per 1,000 wo...   SP.ADO.TFRT   
4        Adolescent fertility rate (births per 1,000 wo...   SP.ADO.TFRT   
...                                                    ...           ...   
1788377  Worried about not having enough money for old ...   fin44a2.d.1   
1788378  Worried about not having enough money for old ...   fin44a2.d.2   
1788379  Worried about not having enough money for old ...     fin44a1.d   
1788380  Worried about not having enough money for old ...   fin44a1.d.1   
1788381  Worried about not having enough money for old ...   fin44a1.d.2   

         Year  IndicatorValue  
0        1992      128.650227  
1        1993      129.800646  
2        1994      128.792351  
3        1995      127.306551  
4        1996      128.239448  
...       ...             ...  
1788377  2021       20.410000  
1788378  2021       20.360000  
1788379  2021       69.210000  
1788380  2021       70.340000  
1788381  2021       68.020000  

[1788382 rows x 6 columns]

In [7]:
from delta import DeltaTable
from pyspark.sql import SparkSession

# Inicialize a sessão Spark
spark = SparkSession.builder.appName("Write to Delta Lake").getOrCreate()

# Caminho da tabela Delta no HDFS
estatPopul_table = "hdfs://hdfs-nn:9000/projeto_EDSTD/silver/estatPopul_table"

# Carregue a tabela Delta Lake existente
delta_table = DeltaTable.forPath(spark, estatPopul_table)

# Seu DataFrame com os novos dados
df = dfEstatisticasPopulacionais

# Escreva o DataFrame na tabela Delta Lake existente
df.write.format("delta").mode("append").save(estatPopul_table)

In [2]:
spark.sql(
    """
    SELECT *
    FROM silver.estatPopul_table
    Where Year = '2010'
    ORDER BY CountryName
    """
).toPandas()

CountryName CountryCode  \
0      Afghanistan         AFG   
1      Afghanistan         AFG   
2      Afghanistan         AFG   
3      Afghanistan         AFG   
4      Afghanistan         AFG   
...            ...         ...   
76809     Zimbabwe         ZWE   
76810     Zimbabwe         ZWE   
76811     Zimbabwe         ZWE   
76812     Zimbabwe         ZWE   
76813     Zimbabwe         ZWE   

                                           IndicatorName      IndicatorCode  \
0      Adolescent fertility rate (births per 1,000 wo...        SP.ADO.TFRT   
1      Antiretroviral therapy coverage for PMTCT (% o...     SH.HIV.PMTC.ZS   
2                   Birth rate, crude (per 1,000 people)     SP.DYN.CBRT.IN   
3      Births attended by skilled health staff (% of ...     SH.STA.BRTC.ZS   
4      Cause of death, by communicable diseases and m...     SH.DTH.COMM.ZS   
...                                                  ...                ...   
76809  Vulnerable employment, total (% of total emplo...     SL.EMP.VULN.ZS   
76810  Wage and salaried workers, female (% of female...  SL.EMP.WORK.FE.ZS   
76811  Wage and salaried workers, male (% of male emp...  SL.EMP.WORK.MA.ZS   
76812  Wage and salaried workers, total (% of total e...     SL.EMP.WORK.ZS   
76813  Women's share of population ages 15+ living wi...  SH.DYN.AIDS.FE.ZS   

       IndicatorValue  Year  
0          120.820000  2010  
1            2.000000  2010  
2           40.602000  2010  
3           34.300000  2010  
4           48.149914  2010  
...               ...   ...  
76809       66.528710  2010  
76810       23.647020  2010  
76811       42.335810  2010  
76812       32.883550  2010  
76813       58.400000  2010  

[76814 rows x 6 columns]

In [ ]:
#############################################################################################################################################################################
####################################################################### SALARIO MINIMO ######################################################################################
#############################################################################################################################################################################

In [11]:
#read hdfs file to dataframe
#
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("HDFSToDataFrame").getOrCreate()

hdfs_file_path = "hdfs://hdfs-nn:9000/projeto_EDSTD/bronze/SalarioMinimo.csv"

dfSalarioMinimo = spark.read.csv(hdfs_file_path, header=True, inferSchema=True)

#eliminar colunas
columns_to_drop = ['SERIES2', 'Flag Codes', 'Codes', 'TIME6', 'PERIOD', 'Unit', 'Reference Period Code', 'Reference Period', 'PowerCode', 'PowerCode Code', 'Flags']
dfSalarioMinimo = dfSalarioMinimo.drop(*columns_to_drop)

#rename colunas
dfSalarioMinimo = dfSalarioMinimo.withColumnRenamed("COUNTRY0", "CountryCode") \
                                 .withColumnRenamed("Country1", "CountryName") \
                                 .withColumnRenamed("Series3", "Series") \
                                 .withColumnRenamed("Pay period", "PayPeriod") \
                                 .withColumnRenamed("Time7", "Year") \
                                 .withColumnRenamed("Unit Code", "UnitCode")   

# Converter StringType para IntegerType
dfSalarioMinimo = dfSalarioMinimo.withColumn("Value", col("Value").cast("double"))
dfSalarioMinimo = dfSalarioMinimo.withColumn("Year", col("Year").cast("integer"))

dfSalarioMinimo.toPandas()

CountryCode CountryName                                    Series  \
0            IRL     Ireland  In 2022 constant prices at 2022 USD PPPs   
1            IRL     Ireland  In 2022 constant prices at 2022 USD PPPs   
2            IRL     Ireland  In 2022 constant prices at 2022 USD PPPs   
3            IRL     Ireland  In 2022 constant prices at 2022 USD PPPs   
4            IRL     Ireland  In 2022 constant prices at 2022 USD PPPs   
...          ...         ...                                       ...   
3103         HRV     Croatia  In 2022 constant prices at 2022 USD PPPs   
3104         HRV     Croatia  In 2022 constant prices at 2022 USD PPPs   
3105         HRV     Croatia  In 2022 constant prices at 2022 USD PPPs   
3106         HRV     Croatia  In 2022 constant prices at 2022 USD PPPs   
3107         HRV     Croatia  In 2022 constant prices at 2022 USD PPPs   

     PayPeriod  Year UnitCode         Value  
0       Annual  2001      USD  16423.708421  
1       Annual  2002      USD  16486.908463  
2       Annual  2003      USD  16665.709948  
3       Annual  2004      USD  17847.836597  
4       Annual  2005      USD  18627.892526  
...        ...   ...      ...           ...  
3103    Annual  2018      USD  12184.282338  
3104    Annual  2019      USD  13178.638357  
3105    Annual  2020      USD  14274.087628  
3106    Annual  2021      USD  14543.411602  
3107    Annual  2022      USD  14494.139017  

[3108 rows x 7 columns]

In [12]:
from delta import DeltaTable
from pyspark.sql import SparkSession

# Inicialize a sessão Spark
spark = SparkSession.builder.appName("Write to Delta Lake").getOrCreate()

# Caminho da tabela Delta no HDFS
salMin_table = "hdfs://hdfs-nn:9000/projeto_EDSTD/silver/salMin_table"

# Carregue a tabela Delta Lake existente
delta_table = DeltaTable.forPath(spark, salMin_table)

# Seu DataFrame com os novos dados
df = dfSalarioMinimo

# Escreva o DataFrame na tabela Delta Lake existente
df.write.format("delta").mode("append").save(salMin_table)

In [16]:
spark.sql(
    """
    SELECT *
    FROM silver.salMin_table
    """
).toPandas()

CountryCode CountryName  \
0           IRL     Ireland   
1           ESP       Spain   
2           AUS   Australia   
3           ESP       Spain   
4           TUR     Türkiye   
..          ...         ...   
139         BGR    Bulgaria   
140         HRV     Croatia   
141         HRV     Croatia   
142         HRV     Croatia   
143         HRV     Croatia   

                                                Series PayPeriod UnitCode  \
0             In 2022 constant prices at 2022 USD PPPs    Annual      USD   
1    In 2022 constant prices at 2022 USD exchange r...    Annual      USD   
2    In 2022 constant prices at 2022 USD exchange r...    Annual      USD   
3             In 2022 constant prices at 2022 USD PPPs    Hourly      USD   
4    In 2022 constant prices at 2022 USD exchange r...    Hourly      USD   
..                                                 ...       ...      ...   
139           In 2022 constant prices at 2022 USD PPPs    Annual      USD   
140  In 2022 constant prices at 2022 USD exchange r...    Hourly      USD   
141           In 2022 constant prices at 2022 USD PPPs    Hourly      USD   
142  In 2022 constant prices at 2022 USD exchange r...    Annual      USD   
143           In 2022 constant prices at 2022 USD PPPs    Annual      USD   

            Value  Year  
0    19288.966790  2015  
1    11095.438024  2015  
2    27671.529286  2015  
3        8.559432  2015  
4        1.597717  2015  
..            ...   ...  
139   6721.364919  2015  
140      2.534552  2015  
141      4.675364  2015  
142   5947.151301  2015  
143  10970.419810  2015  

[144 rows x 7 columns]